In [1]:
#1.1
import requests
import random
import numpy as np
import sys

url_names = 'https://raw.githubusercontent.com/Micniks/Python-Assignments/master/Week11/names_long.txt'

max_cultist_amount = 6666
min_score = 0
max_score = 100

response = requests.get(url_names)
full_read = ''
for line in response:
    decoded_line = line.decode("utf-8")
    full_read = full_read + decoded_line

all_cultist_names = full_read.split('\n');
cultist_names = []
if(len(all_cultist_names) <= max_cultist_amount): 
    cultist_names = all_cultist_names
else:
    cultist_names = all_cultist_names[:max_cultist_amount]

    
#1.2
def recruit_cultist(name):
    cultist = {name: {
    'stealth': random.randint(min_score, max_score)/10, 
    'influence': random.randint(min_score, max_score)/10, 
    'endurance': random.randint(min_score, max_score)/10, 
    'lore': random.randint(min_score, max_score)/10, 
    'economic': random.randint(min_score, max_score)/10, 
    'strength': random.randint(min_score, max_score)/10, 
    'insanity': random.randint(min_score, max_score)/10}}
    return cultist
    
    
#1.3
def make_cult(cultist_names_list):
    cult = {}
    for cultist in cultist_names_list:
        cult.update(recruit_cultist(cultist))
    return cult

cult = make_cult(cultist_names)


#1.4
cult_roles = {
'Priest': {'stealth': -1, 'influence': 3, 'endurance': 2, 'lore': 4, 'economic': 1, 'strength': 0, 'insanity': 5},
'Enforcer': {'stealth': 1, 'influence': 3, 'endurance': 4, 'lore': 0, 'economic': -1, 'strength': 5, 'insanity': 2},
'Assassin': {'stealth': 5, 'influence': -1, 'endurance': 4, 'lore': 1, 'economic': 0, 'strength': 2, 'insanity': 3},
'Recruiter': {'stealth': 1, 'influence': 5, 'endurance': 0, 'lore': 3, 'economic': 2, 'strength': -1, 'insanity': 4},
'Accountant': {'stealth': 2, 'influence': 1, 'endurance': 3, 'lore': 4, 'economic': 5, 'strength': 0, 'insanity': -1},
'Advisor': {'stealth': 2, 'influence': 4, 'endurance': -1, 'lore': 5, 'economic': 3, 'strength': 0, 'insanity': 1}
}


#Support Method for 1.5
def make_cultist_matrix(data):
    stats = ['stealth', 'influence', 'endurance', 'lore', 'economic', 'strength', 'insanity']
    matrix = []
    names = []
    for name, stats_values in data.items():
        matrix.append(list(stats_values[key] for key in stats))
        names.append(name)
    return {'names': names,'matrix': np.array(matrix)}


#1.5
def assign_cult_roles(cult, cult_roles):
    matrix_cultists_info = make_cultist_matrix(cult)
    cultists_names = matrix_cultists_info['names']
    matrix_C = matrix_cultists_info['matrix']
    matrix_roles_info = make_cultist_matrix(cult_roles)
    roles_names = matrix_roles_info['names']
    matrix_R = matrix_roles_info['matrix'].T
    result_values = matrix_C.dot(matrix_R)
    
    #print('Cultist Values:\n', matrix_C, end='\n\n')
    #print('Roles Modificers:\n', matrix_R, end='\n\n')
    #print('Cultist Role Results:\n', result_values, end='\n\n')
    
    result = {}
    for y_idx in range(0, len(result_values)):
        name = cultists_names[y_idx]
        row = result_values[y_idx]
        highest= {'idx': 0, 'value': -sys.maxsize - 1, 'role': ''}
        for x_idx in range(0,len(row)):
            if(highest['value'] < row[x_idx]):
                highest['idx'] = x_idx
                highest['value'] = row[x_idx]
                highest['role'] = roles_names[x_idx]
        result.update({name: highest['role']})
    return result

def print_role_amount(cultists_roles):
    amounts_for_roles = {}
    print('Total:', len(cultists_roles))
    for role in cult_roles:
        amounts_for_roles.update({role: 0})
    for cultist in cultists_roles:
        role = cultists_roles[cultist]
        amounts_for_roles[role] += 1
    for role in amounts_for_roles:
        print(role + ':', amounts_for_roles[role])

culists_roles = assign_cult_roles(cult, cult_roles)
print_role_amount(culists_roles)
print()

Total: 6666
Priest: 901
Enforcer: 1389
Assassin: 1374
Recruiter: 791
Accountant: 1223
Advisor: 988



In [26]:
import csv
import random

culists_roles = assign_cult_roles(cult, cult_roles)

with open('cultists_v2.csv', 'w', newline='') as csvfile:
    c_writer = csv.writer(csvfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    c_writer.writerow(['Cultist', 'Stealth', 'Influence', 'Endurance', 'Lore', 'Economic', 'Strength', 'Insanity', 'Position', 'Age', 'Membership_in_years', 'Contribution_in_dollars', 'Members_recruited', 'Type_of_living_area'])
    for cultist in cult:
        s = cult[cultist]
        stats = {}
        stats['age'] = random.randint(16,80)
        stats['membership'] = random.randint(1,60)
        if(stats['membership'] > (stats['age'] - 15)): 
            stats['membership'] = stats['age'] - 15
        stats['contribution'] = 0
        stats['recruited'] = 0
        for year in range(0,stats['membership']):
            stats['contribution'] += ((random.randint(1,5) * s['economic'] * 10) + (random.randint(1,5) * s['stealth']))
            stats['recruited'] += int((s['influence'] * s['insanity'])/random.randint(25,40))
            if(culists_roles[cultist] == 'Recruiter'):        
                stats['recruited'] += random.randint(1,10)
        stats['contribution'] = int(stats['contribution'])
        stats['live_in_are'] = random.choices(['Rural', 'City', 'Province'], weights=(15, 60, 25), k=1)[0]
        if(random.randint(1,100) == 1):
            null_stat = random.choices(['age', 'live_in_are'], k=1)[0]
            stats[null_stat] = 'NULL'
        c_writer.writerow([cultist, s['stealth'], s['influence'], s['endurance'], s['lore'], s['economic'], s['strength'], s['insanity'], culists_roles[cultist], stats['age'], stats['membership'], stats['contribution'], stats['recruited'], stats['live_in_are']])

In [27]:
import pandas as pd

data = pd.read_csv('cultists_v2.csv')

data

,Cultist,Stealth,Influence,Endurance,Lore,Economic,Strength,Insanity,Position,Age,Membership_in_years,Contribution_in_dollars,Members_recruited,Type_of_living_area
0,Nellie Simpson,7.5,6.0,1.3,3.4,8.5,0.1,9.7,Recruiter,76.0,13,3162.5,78,City
1,Brad Garner,8.6,7.1,5.6,4.3,8.9,8.0,0.9,Accountant,60.0,26,8313.2,0,Province
2,Kristen Watkins,0.1,4.7,2.9,4.6,2.2,5.6,8.6,Priest,77.0,42,2521.6,42,City
3,Marcella Peters,7.1,1.4,4.5,0.5,1.4,4.0,5.8,Assassin,33.0,18,1167.2,0,City
4,Vivian Montgomery,2.3,5.4,5.1,7.5,3.4,1.3,6.0,Priest,40.0,25,3114.4,12,Province
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6661,Vickie Greenaway,1.1,3.2,5.2,9.5,8.2,5.6,0.2,Accountant,74.0,41,9236.2,0,City
6662,Gerard Allison,9.9,8.0,6.0,7.9,3.2,5.5,2.7,Advisor,55.0,40,5116.4,0,Province
6663,Kunal Hawes,9.2,6.7,3.5,7.4,9.2,6.6,1.0,Accountant,68.0,31,10524.8,0,Province
6664,Dotty Tierney,6.7,7.7,7.7,4.0,2.4,7.0,4.1,Enforcer,50.0,16,1294.4,7,Province
